In [1]:
import os
import NN
import utils
from torch import save as save_model
from torch import load as load_model
import config as cfg
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Train and Evaluate a single model

### Setup Parameters

In [2]:
ETF = 'SPY' # 'SPY', 'DIA', 'QQQ'
NNtype = 'MLP' # 'MLP', 'RNN', 'PSN'
params = cfg.train_parameters[ETF][NNtype]
inputs_lag = cfg.SPYfeatures[NNtype] # SPYfeatures, DIAfeatures, QQQfeatures

### Load Dataframes

In [3]:
traindf = utils.load_file(os.path.join("data", ETF, NNtype, "Train.pkl"))
validdf = utils.load_file(os.path.join("data", ETF, NNtype, "Valid.pkl"))
testdf  = utils.load_file(os.path.join("data", ETF, NNtype, "Test.pkl"))

### Initialize DataLoaders

In [4]:
features_col = []
for i in inputs_lag:
    features_col.append("Return_" + str(i))
target_col   = 'Target'

mu, sigma = None, None

trainloader = utils.DataFrame2DataLoader(traindf, features_col, target_col, batch_size=10, normalize=False)
validloader = utils.DataFrame2DataLoader(validdf, features_col, target_col, batch_size=8, normalize=False)
testloader  = utils.DataFrame2DataLoader(testdf, features_col, target_col, batch_size=8, normalize=False)

### Instanciate Model

In [5]:
model = NN.Model(NNtype)
model.setup(params["input_size"], 
            params["hidden_size"],
            params["output_size"],
            1000,#params["epochs"],
            params["optim"],
            params["lr"],
            params["momentum"]
           )

Sequential(
  (0): Linear(in_features=7, out_features=6, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=6, out_features=1, bias=True)
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.003
    momentum: 0.004
    nesterov: False
    weight_decay: 0
)


### Training

In [6]:
model.train(trainloader, validloader)

[00:00] Epoch 1/1000 - Train Loss : 0.034067 / Val Loss : 0.013573
[00:04] Epoch 100/1000 - Train Loss : 0.000144 / Val Loss : 0.000049
[00:08] Epoch 200/1000 - Train Loss : 0.000144 / Val Loss : 0.000049
[00:12] Epoch 300/1000 - Train Loss : 0.000144 / Val Loss : 0.000049
[00:16] Epoch 400/1000 - Train Loss : 0.000144 / Val Loss : 0.000049
[00:19] Epoch 500/1000 - Train Loss : 0.000144 / Val Loss : 0.000049
[00:23] Epoch 600/1000 - Train Loss : 0.000144 / Val Loss : 0.000049
[00:27] Epoch 700/1000 - Train Loss : 0.000144 / Val Loss : 0.000049
[00:31] Epoch 800/1000 - Train Loss : 0.000144 / Val Loss : 0.000049
[00:35] Epoch 900/1000 - Train Loss : 0.000144 / Val Loss : 0.000049
[00:39] Epoch 1000/1000 - Train Loss : 0.000144 / Val Loss : 0.000049
Train MAE : 0.0082 | Train MAPE  : inf | Train RSME : 0.0001 | Train Theil-U 0.9130
Valid MAE : 0.0053 | Valid MAPE  : inf | Valid RSME : 0.0000 | Valid Theil-U 0.8813


### Evaluation

In [7]:
model.evaluate(testloader)

Test MAE : 0.005533 | Test MAPE  : 1.219308 | Test RSME : 0.000055 | Test Theil-U 0.893752


## Train and Evaluate All Models

In [8]:
models = {'SPY':{'MLP':None,'RNN':None,'PSN':None},
          'DIA':{'MLP':None,'RNN':None,'PSN':None},
          'QQQ':{'MLP':None,'RNN':None,'PSN':None},
         }

for ETF in ['SPY', 'DIA', 'QQQ']:
    for NNtype in ['MLP', 'RNN', 'PSN']:
        print("="*10 + ETF + " - " + NNtype + "="*10)
        params = cfg.train_parameters[ETF][NNtype]
        if ETF == "SPY":
            inputs_lag = cfg.SPYfeatures[NNtype]
        elif ETF == "DIA":
            inputs_lag = cfg.DIAfeatures[NNtype]
        elif ETF == "QQQ":
            inputs_lag = cfg.QQQfeatures[NNtype]

        traindf = utils.load_file(os.path.join("data", ETF, NNtype, "Train.pkl"))
        validdf = utils.load_file(os.path.join("data", ETF, NNtype, "Valid.pkl"))
        testdf  = utils.load_file(os.path.join("data", ETF, NNtype, "Test.pkl"))

        features_col = []
        for i in inputs_lag:
            features_col.append("Return_" + str(i))
        target_col   = 'Target'

        mu, sigma = None, None

        trainloader = utils.DataFrame2DataLoader(traindf, features_col, target_col, batch_size=10, normalize=False)
        validloader = utils.DataFrame2DataLoader(validdf, features_col, target_col, batch_size=8, normalize=False)
        testloader  = utils.DataFrame2DataLoader(testdf, features_col, target_col, batch_size=8, normalize=False)

        model = NN.Model(NNtype)
        model.setup(params["input_size"], 
                    params["hidden_size"],
                    params["output_size"],
                    1000,#params["epochs"],
                    params["optim"],
                    params["lr"],
                    params["momentum"]
                   )

        model.train(trainloader, validloader)
        model.evaluate(testloader)
        
        models[ETF][NNtype] = model
        print("\n"*2)

==========SPY - MLP==========
Sequential(
  (0): Linear(in_features=7, out_features=6, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=6, out_features=1, bias=True)
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.003
    momentum: 0.004
    nesterov: False
    weight_decay: 0
)
[00:00] Epoch 1/1000 - Train Loss : 0.008169 / Val Loss : 0.003563
[00:04] Epoch 100/1000 - Train Loss : 0.000137 / Val Loss : 0.000048
[00:08] Epoch 200/1000 - Train Loss : 0.000137 / Val Loss : 0.000048
[00:12] Epoch 300/1000 - Train Loss : 0.000137 / Val Loss : 0.000048
[00:16] Epoch 400/1000 - Train Loss : 0.000137 / Val Loss : 0.000048
[00:20] Epoch 500/1000 - Train Loss : 0.000137 / Val Loss : 0.000048
[00:24] Epoch 600/1000 - Train Loss : 0.000137 / Val Loss : 0.000048
[00:29] Epoch 700/1000 - Train Loss : 0.000137 / Val Loss : 0.000048
[00:33] Epoch 800/1000 - Train Loss : 0.000137 / Val Loss : 0.000048
[00:37] Epoch 900/1000 - Train Loss : 0.000137 / Val Loss : 0.000048
[00:41] Epoch 1000/100

### Save Models

In [9]:
for ETF in ["SPY", "DIA", "QQQ"]:
    for NNtype in ["MLP", "RNN", "PSN"]:
        os.makedirs(os.path.join("models", ETF, NNtype), exist_ok=True)
        save_model(models[ETF][NNtype].model.state_dict(), os.path.join("models", ETF, NNtype, "ckpt.pth"))

### Load Ckpt

In [8]:
ETF = "DIA"
NNtype = "RNN"
params = cfg.train_parameters[ETF][NNtype]
inputs_lag = cfg.DIAfeatures[NNtype]

features_col = []
for i in inputs_lag:
    features_col.append("Return_" + str(i))
target_col   = 'Target'
df  = utils.load_file(os.path.join("data", ETF, NNtype, "Test.pkl"))
dataloader = utils.DataFrame2DataLoader(df, features_col, target_col, batch_size=8, normalize=False)

modelloaded = NN.Model(NNtype)
modelloaded.setup(params["input_size"], 
            params["hidden_size"],
            params["output_size"],
            1000,#params["epochs"],
            params["optim"],
            params["lr"],
            params["momentum"]
           )
modelloaded.model.load_state_dict(load_model(os.path.join("models", ETF, NNtype, "ckpt.pth")))
modelloaded.evaluate(dataloader)

RNN(
  (rnn): RNN(8, 7)
  (fc): Linear(in_features=7, out_features=1, bias=True)
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.005
    momentum: 0.006
    nesterov: False
    weight_decay: 0
)
Test MAE : 0.005624 | Test MAPE  : inf | Test RSME : 0.000054 | Test Theil-U 0.769070


# Table 4

In [29]:
Table4 = pd.DataFrame()

for ETF in ['SPY', 'DIA', 'QQQ']:
    Table = pd.DataFrame({"Statistic":["MAE", "MAPE", "RMSE", "THEIL-U"]})
    for NNtype in ['MLP', 'RNN', 'PSN']:
        params = cfg.train_parameters[ETF][NNtype]
        if ETF == "SPY":
            inputs_lag = cfg.SPYfeatures[NNtype]
        elif ETF == "DIA":
            inputs_lag = cfg.DIAfeatures[NNtype]
        else:
            inputs_lag = cfg.QQQfeatures[NNtype]
        
        features_col = []
        for i in inputs_lag:
            features_col.append("Return_" + str(i))
        target_col   = 'Target'
        df  = utils.load_file(os.path.join("data", ETF, NNtype, "Test.pkl"))
        dataloader = utils.DataFrame2DataLoader(df, features_col, target_col, batch_size=8, normalize=False)
        
        modelloaded = NN.Model(NNtype)
        modelloaded.setup(params["input_size"],
                          params["hidden_size"],
                          params["output_size"],
                          1000,#params["epochs"],
                          params["optim"],
                          params["lr"],
                          params["momentum"])
        modelloaded.model.load_state_dict(load_model(os.path.join("models", ETF, NNtype, "ckpt.pth")))
        modelloaded.evaluate(dataloader)
        
        Table[NNtype] = [modelloaded.testMAE, modelloaded.testMAPE, modelloaded.testRMSE, modelloaded.testTheilU]
        Table["ETF"] = ETF
    
    Table4 = pd.concat([Table4,Table],ignore_index=True)

Table4.set_index(["ETF"], inplace=True)    
Table4.to_csv("table/Table4.csv", index=True)        

Sequential(
  (0): Linear(in_features=7, out_features=6, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=6, out_features=1, bias=True)
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.003
    momentum: 0.004
    nesterov: False
    weight_decay: 0
)
Test MAE : 0.005540 | Test MAPE  : 1.100655 | Test RSME : 0.000054 | Test Theil-U 0.929284
RNN(
  (rnn): RNN(9, 6)
  (fc): Linear(in_features=6, out_features=1, bias=True)
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.003
    momentum: 0.005
    nesterov: False
    weight_decay: 0
)
Test MAE : 0.006091 | Test MAPE  : 2.762408 | Test RSME : 0.000062 | Test Theil-U 0.736161
PSN(
  (fc): Linear(in_features=9, out_features=5, bias=True)
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.4
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)
Test MAE : 0.005535 | Test MAPE  : 1.503290 | Test RSME : 0.000055 | Test Theil-U 0.858845
Sequential(
  (0): Linear(in_features=7, out_features=9, bias=True)
  (1): Sigmoid()
  (2)